In [1]:

import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
FA_10_feat = t.FloatTensor(np.load('../processed/FA_10_feat.npy'))
normed_matrix = t.FloatTensor(np.load('./norm_A.npy'))
in_feat = 10
out_feat = 23691
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

/home/seongha/anaconda3/envs/serec/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [113]:
# In[2]:
class Net(nn.Module):
    def __init__(self, in_feat, out_feat):
      super(Net, self).__init__()

      self.fc1 = nn.Linear(in_feat, out_feat)
      self.dropout1 = nn.Dropout2d(0.25)
      # self.fc2 = nn.Linear(523, 19021)

    def forward(self, x):
      x = self.fc1(x)
      x = t.relu(x)

      # Apply softmax to x
      output = F.softmax(x, dim=1)
      return output


my_nn = Net(in_feat, out_feat).to(device)
print(my_nn)

Net(
  (fc1): Linear(in_features=10, out_features=23691, bias=True)
  (dropout1): Dropout2d(p=0.25, inplace=False)
)


In [7]:
num_items = 23691
from tqdm import tqdm
from scipy import sparse
from torch.utils.data import Dataset, Data, TemporalData
from torch_sparse import SparseTensor
import torch_geometric.transforms as T
AX = t.matmul(normed_matrix, FA_10_feat) ## 23691, 128
class Dataset(Dataset):
  def __init__(self, file, root_dir=None):
    with open(file, 'r') as f:
      self.sessions = f.readlines()[:]
    self.root_dir = root_dir
    self.adj = None

    self.edge_index = [[],[]]
    self.y = []
    for line in self.sessions:
      items = list(map(int, line.split(",")))
      self.edge_index[0].extend(items[:-1])
      self.edge_index[1].extend(items[1:])
      self.y.append(items[-1])
    
    self.edge_index = t.LongTensor(np.array(self.edge_index, dtype=np.float64))
    self.y = t.LongTensor(np.array(self.y, dtype=np.float64))
    print(self.edge_index)
    self.adj = SparseTensor(row=self.edge_index[0], col=self.edge_index[1], sparse_sizes=(num_items, num_items))
    print(self.adj, self.y.shape)

    events = TemporalData(src=t.Tensor(self.edge_index[0],
      dst=t.Tensor(self.edge_index[1]),
    ))

    self.x = FA_10_feat
    self.num_features = self.x.shape[1]
    self.num_classes = 23691
    
def __getitem__(self, idx):
  session = self.edge_index[idx]
  label = self.y
  return session, label


In [8]:
print("generating dataset,,")
trainset = Dataset('./train.txt', )
from torch.utils.data import DataLoader
print("now loader..")
trainloader = DataLoader(trainset, batch_size=16, shuffle=False)


generating dataset,,
tensor([[ 8122,  8122, 13198,  ..., 17238, 23434, 17271],
        [ 8122, 12704, 15717,  ..., 23434, 17271, 14028]])
SparseTensor(row=tensor([    0,     1,     1,  ..., 23690, 23690, 23690]),
             col=tensor([14648,     1,     1,  ..., 23690, 23690, 23690]),
             size=(23691, 23691), nnz=4743820, density=0.85%) torch.Size([1000000])
now loader..


In [43]:
def one_hot(a, num_classes=23691):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [46]:
from torch_geometric.nn import MessagePassing
import torch
import torch.nn.functional as F

from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T

class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(trainset.num_features, 16, cached=True)
        self.conv2 = GCNConv(16, trainset.num_classes, cached=True)

    def forward(self, x, adj_t):
        x = self.conv1(x, adj_t)
        x = F.relu(x)
        x = self.conv2(x, adj_t)
        return F.log_softmax(x, dim=1)

model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# loss_func = nn.MSELoss()

def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.adj.t())
    print(out.shape)
    oh = F.one_hot(data.y, num_classes=23691)
    print(oh.shape)
    loss = F.nll_loss(out, oh)
    loss.backward()
    optimizer.step()
    return float(loss)

for epoch in range(1, 201):
    loss = train(trainset)

torch.Size([23691, 23691])
torch.Size([100, 23691])


ValueError: Expected input batch_size (23691) to match target batch_size (100).

In [93]:
optim = t.optim.SGD(linear_model.parameters(), lr = 0.01, weight_decay = 0)
loss_func = nn.MSELoss()


In [109]:
def train():
  my_nn.train()
  for i, (sess, label) in tqdm(enumerate(trainloader)):
    print("epoch ", i)
    # print(sess.shape, label.shape)
    # sess = t.matmul(sess.to_dense(), AX).to_sparse() ## 23691 23691 23691 * 128 -> 23691 * 128
    # print(sess)
    sess = sess.to(device)
    logit = my_nn(sess)
    # label = t.tensor(label, dtype=t.int64).to(device)
    label = t.from_numpy(one_hot(np.array(label)))
    label = (label).to(device)
    # logit = logit.mean(dim=0)
    # print(logit.shape, label.shape)

    loss = loss_func(logit.to(t.float64), label.to(t.float64))
    # print(loss)
    loss.backward()
    optim.step()

def validate():
  my_nn.eval()
  for i, (sess, label) in tqdm(enumerate(valloader)):
    print("epoch ", i)
    # print(sess.shape, label.shape)
    # sess = t.matmul(sess.to_dense(), AX).to_sparse() ## 23691 23691 23691 * 128 -> 23691 * 128
    # print(sess)
    sess = sess.to(device)
    logit = my_nn(sess)
    # label = t.tensor(label, dtype=t.int64).to(device)
    label_oh = t.from_numpy(one_hot(np.array(label)))
    label_oh = (label_oh).to(device)
    # logit = logit.mean(dim=0)
    # print(logit.shape, label.shape)
    top100, indices = logit.topk(k=100)
    print(indices.shape)
    print(get_mrr(indices.cpu(), label))
    # print(top100, indices)
  

In [101]:
def get_mrr(indices, targets): #Mean Receiprocal Rank --> Average of rank of next item in the session.
    """
    Calculates the MRR score for the given predictions and targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        mrr (float): the mrr score
    """
    tmp = targets.view(-1, 1)
    targets = tmp.expand_as(indices)
    # print(indices[:10,:], targets[:10,:])
    hits = (targets == indices).nonzero()
    # print("hits ", hits)
    ranks = hits[:, -1] + 1
    ranks = ranks.float()
    # print("ranks ", ranks)
    rranks = t.reciprocal(ranks)
    mrr = t.sum(rranks).data / targets.size(0)
    return mrr

In [ ]:
train()

In [110]:
validate()

2it [00:00, 19.29it/s]

epoch  0
torch.Size([16, 100])
tensor(0.)
epoch  1
torch.Size([16, 100])
tensor(0.)
epoch  2
torch.Size([16, 100])
tensor(0.)
epoch  3
torch.Size([16, 100])
tensor(0.)
epoch  4
torch.Size([16, 100])
tensor(0.)


9it [00:00, 25.89it/s]

epoch  5
torch.Size([16, 100])
tensor(0.)
epoch  6
torch.Size([16, 100])
tensor(0.)
epoch  7
torch.Size([16, 100])
tensor(0.)
epoch  8
torch.Size([16, 100])
tensor(0.)
epoch  9
torch.Size([16, 100])
tensor(0.)
epoch  10


16it [00:00, 27.85it/s]

...

tensor(0.0012)
epoch  48
torch.Size([16, 100])
tensor(0.)
epoch  49
torch.Size([16, 100])
tensor(0.)
epoch  50
torch.Size([16, 100])
tensor(0.)
epoch  51
torch.Size([16, 100])
tensor(0.)
epoch  52
torch.Size([16, 100])
tensor(0.)
epoch  53


58it [00:02, 26.58it/s]

torch.Size([16, 100])
tensor(0.)
epoch  54
torch.Size([16, 100])
tensor(0.)
epoch  55
torch.Size([16, 100])
tensor(0.)
epoch  56
torch.Size([16, 100])
tensor(0.)
epoch  57
torch.Size([16, 100])
tensor(0.)
epoch  58
torch.Size([16, 100])
tensor(0.)
epoch  59


63it [00:02, 25.65it/s]

torch.Size([16, 100])
tensor(0.)
epoch  60
torch.Size([16, 100])
tensor(0.)
epoch  61
torch.Size([16, 100])
tensor(0.)
epoch  62
torch.Size([8, 100])
tensor(0.)
